# Read Me — How to Run This Notebook

Follow these steps before running:
- Use kernel: Python 3.10 (aai520-310)
- Create .env at the project root with OPENAI_API_KEY, FRED_API_KEY, and optional NEWSAPI_KEY/NEWS_API_KEY
- Optional: set TICKER to choose the stock symbol (e.g., TICKER=MSFT)
- Run cells top-to-bottom. The final report prints and is saved to memory_log.txt
- Optional: set EXPORT_HTML=1 to export this executed notebook to HTML

# Final Submission — CrewAI Orchestrated Run

This clean notebook runs the multi-agent financial analysis using the project's orchestrator and agents.
- Loads environment from .env at project root
- Validates keys and supports NEWSAPI_KEY/NEWS_API_KEY
- Runs full CrewAI pipeline via `src/orchestrator.py`
- Optional: export to HTML at the end

In [28]:
# Environment setup and imports
import os, sys, subprocess
from pathlib import Path
from dotenv import load_dotenv

# Resolve project root to current working directory of this notebook
PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"

# Ensure src is importable
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print("Project root:", PROJECT_ROOT)
print("Using src path:", SRC_DIR)

# Load .env explicitly from project root
load_dotenv(PROJECT_ROOT / ".env")

# Optional: ensure requirements are present (no-op if already installed)
REQS = PROJECT_ROOT / "requirements.txt"
if REQS.exists() and os.getenv("INSTALL_REQUIREMENTS", "0") == "1":
    print("Installing requirements (optional)...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQS)], check=False)

Project root: /Users/atulaneja/Documents/GitHub_Class/aai-520-final-project
Using src path: /Users/atulaneja/Documents/GitHub_Class/aai-520-final-project/src


## Agent Modules — Sanity Check

This cell verifies that all CrewAI agent modules import correctly and exposes the expected agent instances.

In [29]:
# Agent Modules — Sanity Check
from agents.critic_agent import critic_agent
from agents.earnings_analyst_agent import earnings_analyst
from agents.market_analyst_agent import market_analyst
from agents.news_analyst_agent import news_analyst
from agents.investment_agents import investment_advisor

agents = [
    ("critic_agent", critic_agent),
    ("earnings_analyst", earnings_analyst),
    ("market_analyst", market_analyst),
    ("news_analyst", news_analyst),
    ("investment_advisor", investment_advisor),
]
for name, ag in agents:
    try:
        tools = getattr(ag, "tools", []) or []
        print(f"✓ {name}: role='{ag.role}', tools={len(tools)}")
    except Exception as e:
        print(f"✗ {name}: import/attr error -> {e}")

✓ critic_agent: role='Quality Assurance Critic', tools=0
✓ earnings_analyst: role='Earnings Analyst', tools=3
✓ market_analyst: role='Macroeconomic Analyst', tools=1
✓ news_analyst: role='News Analyst', tools=1
✓ investment_advisor: role='Investment Advisor', tools=0


## Tasks — Sanity Check


In [30]:
# Tasks — Sanity Check
# Import orchestrator to ensure tasks are wired (agents and contexts assigned)
try:
    from orchestrator import run_analysis as _wired  # noqa: F401
except Exception:
    # Wiring will still be safe-checked below
    pass

from tasks.financial_tasks import (
    memory_retrieval_task,
    earnings_analysis_task,
    news_analysis_task,
    market_analysis_task,
    advisory_draft_task,
    report_critique_task,
)

all_tasks = [
    ("memory_retrieval_task", memory_retrieval_task),
    ("earnings_analysis_task", earnings_analysis_task),
    ("news_analysis_task", news_analysis_task),
    ("market_analysis_task", market_analysis_task),
    ("advisory_draft_task", advisory_draft_task),
    ("report_critique_task", report_critique_task),
]

for name, t in all_tasks:
    try:
        desc = (t.description or "").strip().split("\n")[0]
        ctx_raw = getattr(t, "context", None)
        ctx_len = len(ctx_raw) if isinstance(ctx_raw, (list, tuple)) else 0
        ag = getattr(t, "agent", None)
        ag_role = getattr(ag, "role", None) if hasattr(ag, "role") else None
        print(f"✓ {name}: desc='{desc[:80]}', context={ctx_len}, agent={ag_role}")
    except Exception:
        # Suppress noisy errors (e.g., NotSpecified); skip printing
        continue

✓ memory_retrieval_task: desc='Retrieve any past analysis for the stock NVDA from the memory log using the Read', context=0, agent=Earnings Analyst
✓ earnings_analysis_task: desc='Analyze the most recent annual financial statements for the stock NVDA. First, c', context=1, agent=Earnings Analyst
✓ news_analysis_task: desc='Conduct a comprehensive news analysis for the company associated with the ticker', context=0, agent=News Analyst
✓ market_analysis_task: desc='Analyze the current macroeconomic environment. Fetch the latest data for Gross D', context=0, agent=Macroeconomic Analyst
✓ advisory_draft_task: desc='Synthesize the financial statement analysis, news sentiment analysis, and macroe', context=3, agent=Investment Advisor
✓ report_critique_task: desc='Review the DRAFT investment report provided in the context. Your role is to act ', context=1, agent=Quality Assurance Critic


## Validate environment and run orchestration

In [31]:
import os, traceback

# Accept NEWS_API_KEY alias
if "NEWSAPI_KEY" not in os.environ and os.getenv("NEWS_API_KEY"):
    os.environ["NEWSAPI_KEY"] = os.getenv("NEWS_API_KEY")

required_keys = ["OPENAI_API_KEY", "FRED_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    print("Missing required keys:", missing)
    print("Add them to .env and rerun. Skipping orchestrator run.")
else:
    ticker = os.getenv("TICKER", "NVDA")
    try:
        # Orchestrated CrewAI run
        from orchestrator import run_analysis
        print(f"Starting multi-agent analysis for {ticker}...")
        run_analysis(ticker)
    except Exception as e:
        print("[ERROR] Orchestrator run failed:", e)
        traceback.print_exc()

Starting multi-agent analysis for NVDA...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: aa20b51a-1698-4340-a012-f81c8987e8a7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Task: Retrieve any past analysis for the stock NVDA from the memory log using the Read Memory Tool.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Earnings Analyst                                                                                        │
│                                                                                                                 │
│  Thought: I need to check if there are any past analyses for the stock NVDA in the memory log.                  │
│                                                                                                                 │
│  Using Tool: Read Memory Tool                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Critic                                                                                │
│                                                                                                                 │
│  Task: Review the DRAFT investment report provided in the context. Your role is to act as a skeptical quality   │
│  assurance analyst. Critique the report based on the following principles:                                      │
│  1. Clarity and Cohesion: Is the main investment thesis clear and easy to understand?                           │
│  2. Evidentiary Support: Are all claims backed by specific data points from the analyses?                       │
│  3. Risk Assessment: Does the report adequately identify and discuss the primary risks?                         │
│                                                                                                                 │
│  After your critique, produce a FINAL, polished investment report that incorporates your feedback and provides  │
│  a definitive recommendation for NVDA.                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Critic                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Investment Recommendation Report for NVIDIA Corporation (NVDA)**                                             │
│                                                                                                                 │
│  **Recommendation:** Buy                                                                                        │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│                                                                                                                 │
│  **1. Financial Statement Analysis:**                                                                           │
│  NVIDIA's financial statements present a convincing overview of the company's fiscal health and competitive     │
│  positioning within the tech sector. In its latest annual report, NVDA reported total revenue of **$130.497     │
│  billion**, reflecting substantial growth attributed primarily to heightened demand for its GPUs and data       │
│  center products. This robust revenue figure is complemented by a gross profit of **$97.858 billion**,          │
│  resulting in a gross profit margin of **74.94%**. Such a high margin suggests not only effective cost          │
│  management but also strong pricing power, crucial in retaining market leadership.                              │
│                                                                                                                 │
│  The company’s net income stands at **$72.880 billion**, which translates into a remarkable net income margin   │
│  of **55.79%**, placing NVIDIA among the most profitable companies in its sector. The diluted earnings per      │
│  share (EPS) of **$2.94** further indicates significant returns for shareholders.                               │
│                                                                                                                 │
│  NVIDIA's balance sheet reveals considerable strength; total assets amount to **$111.601 billion**, while       │
│  total liabilities are only **$32.274 billion**. This results in an exceptionally low debt to equity ratio of   │
│  **0.13**, highlighting a conservative capital structure that reduces financial risk exposure. Additionally,    │
│  with a working capital of **$62.079 billion**, the company showcases significant liquidity to meet             │
│  operational needs and invest in growth opportunities.                                                          │
│                                                                                                                 │
│  The cash flow statement reinforces NVIDIA's operational strength, reflecting an operating cash flow of         │
│  **$64.089 billion** and free cash flow of **$60.853 billion**. Furthermore, an ending cash position of         │
│  **$8.589 billion** signifies a solid increase from the prior year, underscoring effective cash management      │
│  strategies capable of supporting dividends, share buybacks, or reinvestment into core operations.              │
│                                                                                                                 │
│  **2. News Sentiment Analysis:**                       

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6489642f-5067-483b-889f-4cda50586e73                                                                     │
│  Agent: Quality Assurance Critic                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: aa20b51a-1698-4340-a012-f81c8987e8a7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Investment Recommendation Report for NVIDIA Corporation (NVDA)**                               │
│                                                                                                                 │
│  **Recommendation:** Buy                                                                                        │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│                                                                                                                 │
│  **1. Financial Statement Analysis:**                                                                           │
│  NVIDIA's financial statements present a convincing overview of the company's fiscal health and competitive     │
│  positioning within the tech sector. In its latest annual report, NVDA reported total revenue of **$130.497     │
│  billion**, reflecting substantial growth attributed primarily to heightened demand for its GPUs and data       │
│  center products. This robust revenue figure is complemented by a gross profit of **$97.858 billion**,          │
│  resulting in a gross profit margin of **74.94%**. Such a high margin suggests not only effective cost          │
│  management but also strong pricing power, crucial in retaining market leadership.                              │
│                                                                                                                 │
│  The company’s net income stands at **$72.880 billion**, which translates into a remarkable net income margin   │
│  of **55.79%**, placing NVIDIA among the most profitable companies in its sector. The diluted earnings per      │
│  share (EPS) of **$2.94** further indicates significant returns for shareholders.                               │
│                                                                                                                 │
│  NVIDIA's balance sheet reveals considerable strength; total assets amount to **$111.601 billion**, while       │
│  total liabilities are only **$32.274 billion**. This results in an exceptionally low debt to equity ratio of   │
│  **0.13**, highlighting a conservative capital structure that reduces financial risk exposure. Additionally,    │
│  with a working capital of **$62.079 billion**, the company showcases significant liquidity to meet             │
│  operational needs and invest in growth opportunities.                                                          │
│                                                                                                                 │
│  The cash flow statement reinforces NVIDIA's operational strength, reflecting an operating cash flow of         │
│  **$64.089 billion** and free cash flow of **$60.853 billion**. Furthermore, an ending cash position of         │
│  **$8.589 billion** signifies a solid increase from the prior year, underscoring effective cash management      │
│  strategies capable of supporting dividends, share buybacks, or reinvestment into core operations.              │
│                                                       



########################
## Final Analysis Report:
########################
**Investment Recommendation Report for NVIDIA Corporation (NVDA)**  

**Recommendation:** Buy  

**Rationale:**  

**1. Financial Statement Analysis:**  
NVIDIA's financial statements present a convincing overview of the company's fiscal health and competitive positioning within the tech sector. In its latest annual report, NVDA reported total revenue of **$130.497 billion**, reflecting substantial growth attributed primarily to heightened demand for its GPUs and data center products. This robust revenue figure is complemented by a gross profit of **$97.858 billion**, resulting in a gross profit margin of **74.94%**. Such a high margin suggests not only effective cost management but also strong pricing power, crucial in retaining market leadership.

The company’s net income stands at **$72.880 billion**, which translates into a remarkable net income margin of **55.79%**, placing NVIDIA among the most profitab

## Optional: HTML export of this executed notebook

In [32]:
# Set EXPORT_HTML=1 in environment to enable.
import os
if os.getenv("EXPORT_HTML", "0") == "1":
    try:
        import nbformat, nbconvert
        from nbconvert import HTMLExporter
        from nbconvert.writers import FilesWriter
        import json
        from pathlib import Path
        
        nb_path = Path(__file__).resolve() if '__file__' in globals() else None
        if nb_path is None:
            # When running inside Jupyter UI, provide a default path
            nb_path = Path.cwd() / "Final_Submission_CrewAI_Run.ipynb"
        
        # Best-effort read and export
        with open(nb_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        html_exporter = HTMLExporter()
        (body, resources) = html_exporter.from_notebook_node(nb)
        out_file = nb_path.with_suffix('.html')
        with open(out_file, 'w', encoding='utf-8') as f:
            f.write(body)
        print("Exported HTML:", out_file)
    except Exception as e:
        print("[WARN] HTML export skipped:", e)